<a href="https://www.kaggle.com/code/archismancoder/binarypredpoisonousmushrooms-autogluon?scriptVersionId=191866990" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')
test.head()

In [ ]:
train.info()

In [ ]:
train.nunique().sort_values(ascending=False)

In [ ]:
round(train['class'].value_counts()*100/len(train),2)

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().value_counts()

In [ ]:


import matplotlib.pyplot as plt

missing_values = train.isnull().mean() * 100

# Plot
missing_values.plot(kind='bar', figsize=(8, 4))

plt.title('Percentage of Missing Values by Feature')
plt.ylabel('Percentage')
plt.xlabel('Features')
plt.xticks(rotation=45)
plt.show()



In [ ]:
import seaborn as sns

plt.figure(figsize=(18,12))
plt.title("Visualizing Missing Values")
sns.heatmap(train.isnull(), cbar=False, cmap=sns.color_palette("Spectral_r", n_colors=13), yticklabels=False);

In [ ]:
train[train.duplicated()].head()

In [ ]:
train.drop_duplicates( keep='first', inplace=True)
train.reset_index(inplace=True,drop=True)
train.duplicated().value_counts()

In [ ]:
!pip install ray==2.10.0
!pip install autogluon.tabular
!pip install -U ipywidgets

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
label = 'class'
train[label].describe()

In [ ]:
predictor = TabularPredictor(label=label,eval_metric ='mcc',
                            problem_type="binary").fit(train,presets='best_quality',
                                                                    time_limit=3600*9,verbosity=3,
                                                       excluded_model_types=['KNN'],
                                                       ag_args_fit={'num_gpus': 1}
                                                      )
results = predictor.fit_summary()

In [ ]:
predictor.leaderboard()

In [ ]:
y_pred = predictor.predict(test)
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['class'])
df.head()

In [ ]:
sol=pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')
sol.head()

In [ ]:
sol['class']=df['class']
sol.head()

In [ ]:
sol.to_csv('autogluon_automl_submission.csv',index=False)